In [ ]:
import telnetlib
import functools
import time

def readm27(mess):
        if b"Not SD printing" in mess:
            return "notp",0.0
        #elif b"T:" in mess :
        #    return "heat",0.0
        elif b"SD printing" in mess:
            text=mess.decode("utf-8")
            percentext=""
            for l in text:
                if (l>='0' and l<='9') or l=="/":
                    percentext+=l
            temp=percentext.split("/")
            return "print",100*int(temp[0])/int(temp[1])
        else:
            return "else",0.0
            

def checkanswer(mess):
        if b"Resend" in mess:
            return True
        elif b"ok" in mess:
            return False
        elif b"T:" in mess :
            print("Actualmente la impresora esta calentando e ignorara otros comandos, espere un minuto")
            for n in range(6):
                time.sleep(10)
                print(str((n+1)*10)+" segundos pasados...")
            print("Listo")
            return True
        else:
            return True
def showmenu(oplist):
        print("Seleccione accion deseada:")
        for opn, opt in enumerate(opciones):
            print(str(opn+1)+". "+opt)
        select=input("Ingrese numero opcion: ")
        return select
def parseco(command,cs,ln):
        prefix = "N" + str(ln) + " " + command
        cs=functools.reduce(lambda x, y: x ^ y, map(ord, prefix))
        ln=ln+1
        return prefix + "*" + str(cs), cs, ln
def leefile(filename):
        file = open(filename, 'r') 
        sep = ';'
        comandos=[]
        comandos.append("M28 "+filename)
        for line in file:
            rest = (line[:-1]).split(sep, 1)[0]
            if len(rest)>0:
                if rest[-1]==" ":
                    rest=rest[:-1]
                comandos.append(rest)
        comandos.append("M29 "+filename)
        file.close()
        return comandos
    
cs=0
ln=0
host = "192.168.43.101"
port = "8888"
tn = telnetlib.Telnet(host, port)

print("----------Inicializando----------")
command,cs,ln = parseco("M110",cs,ln)
send=True
while send:
    print(command)
    tn.write((command+"\n").encode('ascii'))
    mess=tn.read_until(b"ok",timeout=3)
    print(mess)
    send=checkanswer(mess)

print("----------Inicializado----------")

print("Bienvenido a la interfaz de control!")
opciones=["Ver SD","Subir a SD","Imprimir","Progreso Impresion","Salir"]
loop=True
#ln+=1
while loop:
    
    sel="none"
    seln=showmenu(opciones)
    print()
    try:
        if (int(seln)<1):
            print("Ingreso invalido!")
        else:
            sel=opciones[int(seln)-1]
    except ValueError:
        print("Ingreso invalido!")
    
    if sel == "Salir":
        print("Saliendo...")
        loop=False
    
    if sel == "Subir a SD":
        print("Subiendo...")
        filnam=input("Ingrese nombre archivo: ")
        comandos=leefile(filnam)
        for rawco in comandos:
            command,cs,ln=parseco(rawco,cs,ln)
            send=True
            while send:
                print(command)
                tn.write((command+"\n").encode('ascii'))
                mess=tn.read_until(b"ok",timeout=10)
                print(mess)
                send=checkanswer(mess)
        print("ARCHIVO "+filnam+" SUBIDO\n")
        
    if sel == "Imprimir":
        filnam=input("Ingrese archivo a imprimir: ")
        comandos=[("M23 "+filnam),"M24"+filnam]
        for rawco in comandos:
            command,cs,ln=parseco(rawco,cs,ln)
            send=True
            while send:
                print(command)
                tn.write((command+"\n").encode('ascii'))
                mess=tn.read_until(b"ok",timeout=10)
                print(mess)
                send=checkanswer(mess)
        print("Imprimiendo\n")
        
    if sel == "Progreso Impresion":
        command,cs,ln=parseco("M27",cs,ln)
        send=True
        while send:
            print(command)
            tn.write((command+"\n").encode('ascii'))
            mess=tn.read_until(b"ok",timeout=10)
            print(mess)
            send=checkanswer(mess)
        est,per=readm27(mess)
        if est == "notp":
            print("No se esta imprimiendo nada\n")
        elif est == "heat":
            print("La impresora se esta calentando\n")
        elif est == "print":
            print("La impresion va al "+str(per)+"%\n")
        else:
            print("La impresora se esta preparando a imprimir\n")
           
    if sel == "Ver SD":
        command,cs,ln=parseco("M20",cs,ln)
        send=True
        while send:
            print(command)
            tn.write((command+"\n").encode('ascii'))
            mess=tn.read_until(b"ok",timeout=10)
            print(mess)
            send=checkanswer(mess)
        text=mess.decode("utf-8")
        print(text[:-2])
        
        
tn.close()
print("Adios")

----------Inicializando----------
N0 M110*35
b'ok'
----------Inicializado----------
Bienvenido a la interfaz de control!
Seleccione accion deseada:
1. Ver SD
2. Subir a SD
3. Imprimir
4. Progreso Impresion
5. Salir
Ingrese numero opcion: 1

N1 M20*16
b'\nBegin file list\nSHORT.GCO 67\r\nTEST.GCO 13924\r\nCUB.GCO 263597\r\nBOCA60~1.GCO 3422947\r\nPI3_CA~1.GCO 3571890\r\nPI3_DO~1.GCO 4852306\r\nPI3_EN~1.GCO 5706852\r\nPI3_PL~1.GCO 251526\r\nPI3_CA~2.GCO 953652\r\nCAJA-O~1.GCO 930214\r\nPI3_CA~3.GCO 288922\r\nPI3_TE~1.GCO 1108006\r\nX10_TE~1.GCO 12011057\r\nPI3_TE~2.GCO 1005444\r\nTENAZA~1.GCO 1000662\r\nX10~1.GCO 22562607\r\nTENSOR~1.GCO 3947750\r\nTOP-60.GCO 2413855\r\nTOP-0.GCO 2360132\r\nEnd file list\nok'

Begin file list
SHORT.GCO 67
TEST.GCO 13924
CUB.GCO 263597
BOCA60~1.GCO 3422947
PI3_CA~1.GCO 3571890
PI3_DO~1.GCO 4852306
PI3_EN~1.GCO 5706852
PI3_PL~1.GCO 251526
PI3_CA~2.GCO 953652
CAJA-O~1.GCO 930214
PI3_CA~3.GCO 288922
PI3_TE~1.GCO 1108006
X10_TE~1.GCO 12011057
PI3_TE~2.GCO 100

Ingrese numero opcion: 4

N9 M27*31
b'\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\nSD printing byte 8599/13924\r\nok'
La impresion va al 61.75667911519678%

Seleccione accion deseada:
1. Ver SD
2. Subir a SD
3. Imprimir
4. Progreso Impresion
5. Salir
Ingrese numero opcion: 4

N10 M27*39
b'\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extrusion prevented\necho: cold extr

In [5]:
tn.close()